In [79]:
from dask.distributed import Client
import coiled
from dask.distributed import progress
import io
import sqlalchemy as sa
import urllib
import pandas as pd

import dask.dataframe as dd

# Local
from dask.diagnostics import ProgressBar
ProgressBar().register()

# Cluster

pd.set_option('max_columns', None)


# Connects to the existing "production" cluster
cluster = coiled.Cluster(
    n_workers=20
)
client = Client(cluster)
cluster

coiled.Cluster(Santhin-de94520a-d, 'tls://ec2-18-221-107-218.us-east-2.compute.amazonaws.com:8786', workers=0, threads=0, memory=0 B)

In [2]:
# Big brain with dask
# from dask.diagnostics import ProgressBar
# ProgressBar().register()
# server = 'analityk.wwsi.edu.pl,50221'
# database = 'Synop'
# username = 'psierkin'
# password = 'Qazwsx123456!@#'
# conn= urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+str(server)+';DATABASE='+str(database)+';UID='+str(username)+';PWD='+ str(password))
# engine = sa.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conn),fast_executemany=True)
# query_synop_dask ="""
# SELECT d.Stacja as IDStacji_synop,s.Nazwa,s.Kraj, d.Data, d.Godzina
# 		      ,d.[KierunekWiatru]
#       ,d.[PredkoscWiatru]
#       ,d.[TemperaturaPowietrza]
#       ,d.[TemperaturaPunktuRosy]
#       ,d.[CisnienieNaPoziomieStacji]
#       ,d.[CisnienieNaPoziomieMorza]
#       ,d.[WielkoscTendencjiCisnienia]
#       ,d.[WysokoscOpadu]
#       ,d.[OkresPomiaruOpadu]
# from Depesze d join Stacje as s on d.idstacji=s.IDStacji
#   where s.Kraj in ('Germany','Poland','Belarus','Czech Republic','Lithuania','Slovakia','Ukraine') and d.Data>='2000-01-01'
#   order by d.Data asc
# """
#ddf = dd.from_pandas(pd.read_sql(query_synop_dask, engine), npartitions=20)


#ddf = dd.read_csv('./data/synop_02.02.csv',sep=';',decimal=',',assume_missing=True)
ddf = dd.read_csv('s3://smogoliczka/synop_02.02.csv', sep=';',
                  decimal=',', assume_missing=True, storage_options={'anon': True})

In [3]:
ddf = ddf[ddf['Kraj'] == 'Poland']
ddf = ddf.drop(columns=['Nazwa', 'Kraj'])


def f(df: pd.DataFrame) -> pd.DataFrame:
    """ This is your code from above, as a function """
    #df['Czas pomiaru'] = df['Data'].apply(pd.to_datetime) + pd.to_timedelta(df['Godzina'].astype(str))
    df['Czas pomiaru'] = pd.to_datetime(df.Data) + pd.to_timedelta(df.Godzina)
    df['Czas pomiaru'] = df['Czas pomiaru'].dt.floor('h')
    #df['Czas pomiaru_v2']=pd.to_datetime(df['Czas pomiaru_v2'], errors='coerce', format="%Y-%m-%d %H")
    return df


columns = names = ["IDStacji_synop", 'Data', 'Godzina', "KierunekWiatru",
                   "PredkoscWiatru", "TemperaturaPowietrza", "TemperaturaPunktuRosy",
                   "CisnienieNaPoziomieStacji", "CisnienieNaPoziomieMorza",
                   "WielkoscTendencjiCisnienia", "WysokoscOpadu", "OkresPomiaruOpadu", "Czas pomiaru"
                   ]
df = pd.DataFrame(columns=columns)
ddf = ddf.map_partitions(f, meta=df).persist()
ddf['IDStacji_synop'] = ddf['IDStacji_synop'].astype('int64')
ddf['Czas pomiaru'] = ddf['Czas pomiaru'].astype('datetime64')

In [4]:
ddf.head()

,IDStacji_synop,Data,Godzina,KierunekWiatru,PredkoscWiatru,TemperaturaPowietrza,TemperaturaPunktuRosy,CisnienieNaPoziomieStacji,CisnienieNaPoziomieMorza,WielkoscTendencjiCisnienia,WysokoscOpadu,OkresPomiaruOpadu,Czas pomiaru
0,12295,2000-01-01,16:00:00,270.0,2.0,0.0,-1.3,1004.7,1023.9,0.3,0.0,0.0,2000-01-01 16:00:00
1,12280,2000-01-01,16:00:00,230.0,1.0,-0.1,-0.9,1006.9,1023.6,0.1,0.0,0.0,2000-01-01 16:00:00
2,12272,2000-01-01,16:00:00,220.0,1.0,-0.7,-1.8,1006.3,1023.7,0.1,0.0,0.0,2000-01-01 16:00:00
3,12270,2000-01-01,16:00:00,240.0,3.0,-1.7,-2.0,1005.4,1024.4,0.0,0.0,0.0,2000-01-01 16:00:00
4,12250,2000-01-01,16:00:00,150.0,2.0,-1.0,-2.1,1014.5,1023.7,0.4,0.0,0.0,2000-01-01 16:00:00


In [5]:
helper=ddf[["IDStacji_synop","Czas pomiaru"]].copy()

In [6]:
helper.head()

,IDStacji_synop,Czas pomiaru
0,12295,2000-01-01 16:00:00
1,12280,2000-01-01 16:00:00
2,12272,2000-01-01 16:00:00
3,12270,2000-01-01 16:00:00
4,12250,2000-01-01 16:00:00


In [7]:
client.rebalance(ddf)
client.rebalance(helper)

In [8]:
# import os
# from dotenv import load_dotenv, find_dotenv

# load_dotenv(find_dotenv())

# SERVER = os.environ.get("SERVER")
# DATABASE = os.environ.get("DATABASE")

# USERNAME = os.environ.get("USERNAME")
# PASSWORD = os.environ.get("PASSWORD")
# conn= urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+str(server)+';DATABASE='+str(database)+';UID='+str(username)+';PWD='+ str(password))
# engine = sa.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conn),fast_executemany=True)
# query_synop_smog_pomiary_pivoted= """
#   select pp.[Kod stacji_metadane]
#       ,pp.[Czas pomiaru]
#       ,pp.[C6H6]
#       ,pp.[CO]
#       ,pp.[NO2]
#       ,pp.[O3]
#       ,pp.[PM10]
#       ,pp.[PM2.5]
#       ,pp.[SO2]
# 	  ,ss.station_1_id
# 	  ,ss.station_2_id
# 	  ,ss.station_3_id
#   from pomiary_pivoted as pp join [synop_smogv2] as ss on pp.[Kod stacji_metadane]=ss.KodStacji
#   order by pp.[Czas pomiaru] asc
#   """
# ddf2 = dd.from_pandas(pd.read_sql(query_synop_smog_pomiary_pivoted_synop_smogv2, engine), npartitions=20)

ddf2 = dd.read_csv('s3://smogoliczka/smogoliczkaarchive_15.02.csv', sep=';',
                   decimal=',', assume_missing=True, storage_options={'anon': True})


# local file to cluster
# https://stackoverflow.com/questions/43796774/loading-local-file-from-client-onto-dask-distributed-cluster
# ddf2 = pd.read_csv('./data/smogoliczkaarchive_15.02.csv',sep=';',decimal=',')
# future = client.scatter(ddf2)
# import dask.dataframe as dd
# ddf2 = dd.from_delayed([future], meta=ddf2)  # build dask.dataframe on remote data
# ddf2 = ddf2.repartition(npartitions=10).persist()
# client.rebalance(ddf2)  #

In [9]:
ddf2[[col for col in ddf2.columns if col.startswith(
    "station_")]] = ddf2.loc[:, ddf2.columns.str.startswith("station_")].astype('int64')
ddf2['Czas pomiaru'] = ddf2['Czas pomiaru'].astype('datetime64')
ddf2['Czas pomiaru'] = ddf2['Czas pomiaru'].dt.floor('h')

In [10]:
ddf2.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,station_1_id,station_2_id,station_3_id,station_4_id,station_5_id,station_6_id
0,PmGdaWyzwo03,2000-01-01 01:00:00,NaN,NaN,NaN,38.0,NaN,NaN,NaN,12140,12150,12145,12142,12155,12146
1,PmGdaPoWie01,2000-01-01 01:00:00,NaN,NaN,25.0,NaN,NaN,NaN,NaN,12140,12150,12145,12155,12146,12142
2,DsJelw05,2000-01-01 02:00:00,NaN,NaN,14.0,29.0,NaN,NaN,NaN,12405,12500,12510,12415,12400,12312
3,MpKrakAlKras,2000-01-01 02:00:00,NaN,NaN,62.0,NaN,NaN,NaN,NaN,12566,12560,12600,12660,12575,12555
4,SlKatoKossut,2000-01-01 02:00:00,NaN,NaN,NaN,11.0,NaN,NaN,NaN,12560,12555,12600,12540,12550,12566


In [11]:
ddf3 = ddf2.merge(helper, left_on=['station_1_id', 'Czas pomiaru'], right_on=[
                  'IDStacji_synop', 'Czas pomiaru'], how='left')
ddf3 = client.persist(ddf3)

In [12]:
ddf3.columns

Index(['Kod stacji_metadane', 'Czas pomiaru', 'C6H6', 'CO', 'NO2', 'O3',
       'PM10', 'PM2.5', 'SO2', 'station_1_id', 'station_2_id', 'station_3_id',
       'station_4_id', 'station_5_id', 'station_6_id', 'IDStacji_synop'],
      dtype='object')

In [13]:
ddf3.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,station_1_id,station_2_id,station_3_id,station_4_id,station_5_id,station_6_id,IDStacji_synop
0,SlKatoKossut,2000-01-01 02:00:00,NaN,NaN,NaN,11.0,NaN,NaN,NaN,12560,12555,12600,12540,12550,12566,12560.0
1,SlUstroCies_sanat,2000-01-01 02:00:00,NaN,NaN,NaN,19.0,NaN,NaN,NaN,12600,12540,12560,12566,12555,12625,12600.0
2,MzWarszKrucza,2000-01-01 02:00:00,NaN,NaN,NaN,12.0,NaN,NaN,NaN,12376,12372,12375,12374,12365,12471,NaN
3,SlSosnoSosn_narut,2000-01-01 02:00:00,NaN,NaN,55.0,NaN,NaN,NaN,NaN,12560,12555,12566,12600,12550,12540,12560.0
4,MzWarPodlesn,2000-01-01 02:00:00,NaN,NaN,NaN,9.0,NaN,NaN,NaN,12376,12372,12374,12375,12365,12471,NaN


In [14]:
missing_values = ddf3['IDStacji_synop'].isnull().sum()
missing_count = ((missing_values / ddf3.index.size * 100))
with ProgressBar():
    missing_count_pct = missing_count.compute()
print(missing_count_pct)
print(missing_values.compute())

36.934771622353786
8820552


In [15]:
ddf4 = ddf3.merge(helper, left_on=['station_2_id', 'Czas pomiaru'], right_on=[
                  'IDStacji_synop', 'Czas pomiaru'], how='left')
ddf4 = client.persist(ddf4)
ddf4.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,station_1_id,station_2_id,station_3_id,station_4_id,station_5_id,station_6_id,IDStacji_synop_x,IDStacji_synop_y
0,MzWarszKrucza,2000-01-04 08:00:00,NaN,NaN,NaN,8.0,NaN,NaN,NaN,12376,12372,12375,12374,12365,12471,NaN,NaN
1,MzWarPodlesn,2000-01-04 08:00:00,NaN,NaN,NaN,11.0,NaN,NaN,NaN,12376,12372,12374,12375,12365,12471,NaN,NaN
2,WpKoniWyszyn,2000-01-05 20:00:00,NaN,NaN,NaN,60.0,NaN,NaN,NaN,12345,12336,12435,12342,12452,12326,NaN,NaN
3,LbJarczWolaM,2000-01-11 20:00:00,NaN,NaN,NaN,47.0,NaN,NaN,NaN,12490,12488,12491,12385,12485,12495,NaN,12488.0
4,PmSopBitPl06,2000-01-13 02:00:00,NaN,NaN,22.0,NaN,NaN,NaN,NaN,12150,12145,12140,12142,12135,12155,NaN,NaN


In [16]:
missing_values = ddf4['IDStacji_synop_y'].isnull().sum()
missing_count = ((missing_values / ddf4.index.size * 100))
with ProgressBar():
    missing_count_pct = missing_count.compute()
print(missing_count_pct)
print(missing_values.compute())

43.41010349299537
10367286


In [17]:
ddf4['IDStacji_synop_x'] = ddf4['IDStacji_synop_x'].fillna(
    ddf4['IDStacji_synop_y'])

In [18]:
missing_values = ddf4['IDStacji_synop_x'].isnull().sum()
missing_count = ((missing_values / ddf4.index.size * 100))
with ProgressBar():
    missing_count_pct = missing_count.compute()
print(missing_count_pct)
print(missing_values.compute())

24.114629157218207
5759103


In [19]:
ddf4=client.persist(ddf4)
ddf4.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,station_1_id,station_2_id,station_3_id,station_4_id,station_5_id,station_6_id,IDStacji_synop_x,IDStacji_synop_y
0,MzWarszKrucza,2000-01-04 08:00:00,NaN,NaN,NaN,8.0,NaN,NaN,NaN,12376,12372,12375,12374,12365,12471,NaN,NaN
1,MzWarPodlesn,2000-01-04 08:00:00,NaN,NaN,NaN,11.0,NaN,NaN,NaN,12376,12372,12374,12375,12365,12471,NaN,NaN
2,WpKoniWyszyn,2000-01-05 20:00:00,NaN,NaN,NaN,60.0,NaN,NaN,NaN,12345,12336,12435,12342,12452,12326,NaN,NaN
3,LbJarczWolaM,2000-01-11 20:00:00,NaN,NaN,NaN,47.0,NaN,NaN,NaN,12490,12488,12491,12385,12485,12495,12488.0,12488.0
4,PmSopBitPl06,2000-01-13 02:00:00,NaN,NaN,22.0,NaN,NaN,NaN,NaN,12150,12145,12140,12142,12135,12155,NaN,NaN


In [20]:

ddf5=ddf4.merge(helper, left_on=['station_3_id','Czas pomiaru'],right_on=['IDStacji_synop','Czas pomiaru'],how='left')
ddf5=client.persist(ddf5)
ddf5.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,station_1_id,station_2_id,station_3_id,station_4_id,station_5_id,station_6_id,IDStacji_synop_x,IDStacji_synop_y,IDStacji_synop
0,SlUstroCies_sanat,2000-09-14 06:00:00,NaN,NaN,NaN,41.0,NaN,NaN,NaN,12600,12540,12560,12566,12555,12625,12600.0,12540.0,12560.0
1,MzWarszKrucza,2000-12-31 20:00:00,NaN,NaN,NaN,24.0,NaN,NaN,NaN,12376,12372,12375,12374,12365,12471,NaN,NaN,12375.0
2,DsCzerStraza,2001-07-08 17:00:00,NaN,NaN,5.0,87.0,NaN,NaN,2.0,12405,12510,12500,12415,12520,12424,12510.0,12510.0,12500.0
3,DsJelw05,2001-07-08 17:00:00,NaN,NaN,6.0,97.0,NaN,NaN,10.0,12405,12500,12510,12415,12400,12312,12500.0,12500.0,12510.0
4,SlSosnoSosn_narut,2002-03-23 15:00:00,NaN,NaN,29.0,NaN,NaN,NaN,NaN,12560,12555,12566,12600,12550,12540,12560.0,NaN,12566.0


In [21]:
missing_values = ddf5['IDStacji_synop'].isnull().sum()
missing_count = ((missing_values / ddf5.index.size * 100))
with ProgressBar():
    missing_count_pct = missing_count.compute()
print(missing_count_pct)

39.606095305471214


In [22]:
ddf5['IDStacji_synop_x'] = ddf5['IDStacji_synop_x'].fillna(
    ddf5['IDStacji_synop'])
ddf5 = client.persist(ddf5)
missing_values = ddf5['IDStacji_synop_x'].isnull().sum()
missing_count = ((missing_values / ddf5.index.size * 100))
with ProgressBar():
    missing_count_pct = missing_count.compute()
print(missing_count_pct)

19.49253436623384


In [23]:
ddf5=ddf5.drop(columns=['IDStacji_synop_y','IDStacji_synop'])

In [24]:
ddf5.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,station_1_id,station_2_id,station_3_id,station_4_id,station_5_id,station_6_id,IDStacji_synop_x
0,SlUstroCies_sanat,2000-09-14 06:00:00,NaN,NaN,NaN,41.0,NaN,NaN,NaN,12600,12540,12560,12566,12555,12625,12600.0
1,MzWarszKrucza,2000-12-31 20:00:00,NaN,NaN,NaN,24.0,NaN,NaN,NaN,12376,12372,12375,12374,12365,12471,12375.0
2,DsCzerStraza,2001-07-08 17:00:00,NaN,NaN,5.0,87.0,NaN,NaN,2.0,12405,12510,12500,12415,12520,12424,12510.0
3,DsJelw05,2001-07-08 17:00:00,NaN,NaN,6.0,97.0,NaN,NaN,10.0,12405,12500,12510,12415,12400,12312,12500.0
4,SlSosnoSosn_narut,2002-03-23 15:00:00,NaN,NaN,29.0,NaN,NaN,NaN,NaN,12560,12555,12566,12600,12550,12540,12560.0


In [25]:
ddf6=ddf5.merge(helper, left_on=['station_4_id','Czas pomiaru'],right_on=['IDStacji_synop','Czas pomiaru'],how='left')
ddf6=client.persist(ddf6)
ddf6.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,station_1_id,station_2_id,station_3_id,station_4_id,station_5_id,station_6_id,IDStacji_synop_x,IDStacji_synop
0,SlBytomByto_modrz,2003-05-26 03:00:00,NaN,NaN,17.0,NaN,43.0,NaN,16.0,12560,12555,12550,12540,12600,12566,12560.0,12540.0
1,SlPiekaPiek_darwi,2003-05-26 03:00:00,NaN,0.65,NaN,NaN,NaN,NaN,18.0,12555,12560,12550,12540,12600,12566,12560.0,12540.0
2,OpKKozBSmial,2008-08-08 21:00:00,0.0,0.16,10.0,38.0,19.0,NaN,0.0,12540,12530,12560,12555,12550,12600,12540.0,NaN
3,OpZdziePiast,2008-08-08 21:00:00,2.0,NaN,13.0,NaN,15.0,NaN,0.0,12530,12540,12560,12555,12550,12455,12530.0,NaN
4,PkJasloWIOSFlor,2009-02-02 02:00:00,NaN,NaN,17.5,27.9,NaN,NaN,NaN,12670,12575,12580,12660,12690,12695,NaN,NaN


In [26]:
missing_values = ddf6['IDStacji_synop'].isnull().sum()
missing_count = ((missing_values / ddf6.index.size * 100))
with ProgressBar():
    missing_count_pct = missing_count.compute()
print(missing_count_pct)

35.975150829411035


In [27]:
ddf6['IDStacji_synop_x']=ddf6['IDStacji_synop_x'].fillna(ddf6['IDStacji_synop'])
ddf6=ddf6.drop(columns=['IDStacji_synop'])
missing_values = ddf6['IDStacji_synop_x'].isnull().sum()
missing_count = ((missing_values / ddf6.index.size * 100))
with ProgressBar():
    missing_count_pct = missing_count.compute()
print(missing_count_pct)

15.920889253764539


In [28]:
ddf6.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,station_1_id,station_2_id,station_3_id,station_4_id,station_5_id,station_6_id,IDStacji_synop_x
0,SlBytomByto_modrz,2003-05-26 03:00:00,NaN,NaN,17.0,NaN,43.0,NaN,16.0,12560,12555,12550,12540,12600,12566,12560.0
1,SlPiekaPiek_darwi,2003-05-26 03:00:00,NaN,0.65,NaN,NaN,NaN,NaN,18.0,12555,12560,12550,12540,12600,12566,12560.0
2,OpKKozBSmial,2008-08-08 21:00:00,0.0,0.16,10.0,38.0,19.0,NaN,0.0,12540,12530,12560,12555,12550,12600,12540.0
3,OpZdziePiast,2008-08-08 21:00:00,2.0,NaN,13.0,NaN,15.0,NaN,0.0,12530,12540,12560,12555,12550,12455,12530.0
4,PkJasloWIOSFlor,2009-02-02 02:00:00,NaN,NaN,17.5,27.9,NaN,NaN,NaN,12670,12575,12580,12660,12690,12695,NaN


In [29]:
ddf7=ddf6.merge(helper, left_on=['station_5_id','Czas pomiaru'],right_on=['IDStacji_synop','Czas pomiaru'],how='left')
ddf7=client.persist(ddf7)
ddf7.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,station_1_id,station_2_id,station_3_id,station_4_id,station_5_id,station_6_id,IDStacji_synop_x,IDStacji_synop
0,PmGdaWyzwo03,2006-05-06 19:00:00,NaN,0.309,7.8000,NaN,52.20,NaN,1.7000,12140,12150,12145,12142,12155,12146,NaN,NaN
1,PmGdaLeczk08,2006-05-06 19:00:00,NaN,0.308,9.2000,143.80000,74.10,NaN,2.0000,12140,12150,12145,12142,12155,12146,NaN,NaN
2,MpSzymbaGorl,2020-03-14 10:00:00,NaN,NaN,1.7495,76.14060,NaN,NaN,2.3904,12660,12575,12670,12580,12690,12625,12660.0,12690.0
3,KpToruDziewu,2014-09-01 04:00:00,NaN,NaN,NaN,34.85845,13.50,7.93,NaN,12250,12342,12240,12360,12336,12345,12250.0,12336.0
4,KpToruKaszow,2014-09-01 04:00:00,0.032835,0.190,12.0000,NaN,15.76,NaN,1.6000,12250,12342,12240,12336,12360,12345,12250.0,12360.0


In [30]:
missing_values = ddf7['IDStacji_synop'].isnull().sum()
missing_count = ((missing_values / ddf7.index.size * 100))
with ProgressBar():
    missing_count_pct = missing_count.compute()
print(missing_count_pct)

40.108003986154465


In [31]:
ddf7['IDStacji_synop_x']=ddf7['IDStacji_synop_x'].fillna(ddf7['IDStacji_synop'])
ddf7=ddf7.drop(columns=['IDStacji_synop'])
missing_values = ddf7['IDStacji_synop_x'].isnull().sum()
missing_count = ((missing_values / ddf7.index.size * 100))
with ProgressBar():
    missing_count_pct = missing_count.compute()
print(missing_count_pct)

14.783454863147808


In [32]:
ddf8=ddf7.merge(helper, left_on=['station_6_id','Czas pomiaru'],right_on=['IDStacji_synop','Czas pomiaru'],how='left')
ddf8=client.persist(ddf8)
ddf8.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,station_1_id,station_2_id,station_3_id,station_4_id,station_5_id,station_6_id,IDStacji_synop_x,IDStacji_synop
0,LdGajewUjWod,2018-06-14 15:00:00,NaN,NaN,3.40000,98.00,17.00000,NaN,1.3000,12452,12345,12465,12360,12454,12466,12345.0,NaN
1,SkMalogCemen,2003-07-02 15:00:00,NaN,NaN,6.00000,NaN,39.80000,NaN,25.0000,12570,12469,12550,12466,12485,12566,12570.0,12566.0
2,LdZgieMielcz,2014-12-02 22:00:00,NaN,0.232,17.72204,NaN,43.01138,33.06983,9.5899,12465,12452,12454,12466,12469,12345,12465.0,12345.0
3,KpZielBoryTu,2005-02-28 04:00:00,NaN,NaN,1.00000,59.00,NaN,NaN,0.0000,12235,12240,12215,12250,12136,12150,12235.0,NaN
4,MzGranicaKPN,2014-08-07 21:00:00,NaN,NaN,4.97000,39.73,NaN,NaN,1.5000,12365,12376,12372,12374,12375,12471,12376.0,NaN


In [33]:
missing_values = ddf8['IDStacji_synop'].isnull().sum()
missing_count = ((missing_values / ddf8.index.size * 100))
with ProgressBar():
    missing_count_pct = missing_count.compute()
print(missing_count_pct)

44.42022561429121


In [34]:
ddf8['IDStacji_synop_x']=ddf8['IDStacji_synop_x'].fillna(ddf8['IDStacji_synop'])
ddf8=ddf8.drop(columns=['IDStacji_synop'])
missing_values = ddf8['IDStacji_synop_x'].isnull().sum()
missing_count = ((missing_values / ddf8.index.size * 100))
with ProgressBar():
    missing_count_pct = missing_count.compute()
print(missing_count_pct)
print(missing_values.compute())

13.728220793784335
3279048


In [35]:
ddf8['IDStacji_synop_x'].isnull().sum().compute()

3279048

In [36]:
ddf8.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,station_1_id,station_2_id,station_3_id,station_4_id,station_5_id,station_6_id,IDStacji_synop_x
0,LdGajewUjWod,2018-06-14 15:00:00,NaN,NaN,3.40000,98.00,17.00000,NaN,1.3000,12452,12345,12465,12360,12454,12466,12345.0
1,SkMalogCemen,2003-07-02 15:00:00,NaN,NaN,6.00000,NaN,39.80000,NaN,25.0000,12570,12469,12550,12466,12485,12566,12570.0
2,LdZgieMielcz,2014-12-02 22:00:00,NaN,0.232,17.72204,NaN,43.01138,33.06983,9.5899,12465,12452,12454,12466,12469,12345,12465.0
3,KpZielBoryTu,2005-02-28 04:00:00,NaN,NaN,1.00000,59.00,NaN,NaN,0.0000,12235,12240,12215,12250,12136,12150,12235.0
4,MzGranicaKPN,2014-08-07 21:00:00,NaN,NaN,4.97000,39.73,NaN,NaN,1.5000,12365,12376,12372,12374,12375,12471,12376.0


In [37]:
ddf8.dtypes

Kod stacji_metadane            object
Czas pomiaru           datetime64[ns]
C6H6                          float64
CO                            float64
NO2                           float64
O3                            float64
PM10                          float64
PM2.5                         float64
SO2                           float64
station_1_id                    int64
station_2_id                    int64
station_3_id                    int64
station_4_id                    int64
station_5_id                    int64
station_6_id                    int64
IDStacji_synop_x                int64
dtype: object

In [38]:
ddf8.columns

Index(['Kod stacji_metadane', 'Czas pomiaru', 'C6H6', 'CO', 'NO2', 'O3',
       'PM10', 'PM2.5', 'SO2', 'station_1_id', 'station_2_id', 'station_3_id',
       'station_4_id', 'station_5_id', 'station_6_id', 'IDStacji_synop_x'],
      dtype='object')

In [39]:
ddf8=ddf8.drop(columns=['station_1_id', 'station_2_id', 'station_3_id',
      'station_4_id', 'station_5_id', 'station_6_id'])
ddf8=ddf8.rename(columns={"IDStacji_synop_x": "IDStacji_synop"})

In [40]:
ddf8.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,IDStacji_synop
0,LdGajewUjWod,2018-06-14 15:00:00,NaN,NaN,3.40000,98.00,17.00000,NaN,1.3000,12345.0
1,SkMalogCemen,2003-07-02 15:00:00,NaN,NaN,6.00000,NaN,39.80000,NaN,25.0000,12570.0
2,LdZgieMielcz,2014-12-02 22:00:00,NaN,0.232,17.72204,NaN,43.01138,33.06983,9.5899,12465.0
3,KpZielBoryTu,2005-02-28 04:00:00,NaN,NaN,1.00000,59.00,NaN,NaN,0.0000,12235.0
4,MzGranicaKPN,2014-08-07 21:00:00,NaN,NaN,4.97000,39.73,NaN,NaN,1.5000,12376.0


In [41]:
import numpy as np


ddf8['IDStacji_synop']=ddf8['IDStacji_synop'].fillna(0)
ddf8['IDStacji_synop']=ddf8['IDStacji_synop'].astype('int64')

In [42]:
ddf8=client.persist(ddf8)
ddf8.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,IDStacji_synop
0,LdGajewUjWod,2018-06-14 15:00:00,NaN,NaN,3.40000,98.00,17.00000,NaN,1.3000,12345
1,SkMalogCemen,2003-07-02 15:00:00,NaN,NaN,6.00000,NaN,39.80000,NaN,25.0000,12570
2,LdZgieMielcz,2014-12-02 22:00:00,NaN,0.232,17.72204,NaN,43.01138,33.06983,9.5899,12465
3,KpZielBoryTu,2005-02-28 04:00:00,NaN,NaN,1.00000,59.00,NaN,NaN,0.0000,12235
4,MzGranicaKPN,2014-08-07 21:00:00,NaN,NaN,4.97000,39.73,NaN,NaN,1.5000,12376


In [43]:
ddf8.dtypes

Kod stacji_metadane            object
Czas pomiaru           datetime64[ns]
C6H6                          float64
CO                            float64
NO2                           float64
O3                            float64
PM10                          float64
PM2.5                         float64
SO2                           float64
IDStacji_synop                  int64
dtype: object

In [44]:
ddf9=ddf8.merge(ddf, left_on=['IDStacji_synop','Czas pomiaru'],right_on=['IDStacji_synop','Czas pomiaru'],how='left')
ddf9=client.persist(ddf9)

In [45]:
ddf9.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,IDStacji_synop,Data,Godzina,KierunekWiatru,PredkoscWiatru,TemperaturaPowietrza,TemperaturaPunktuRosy,CisnienieNaPoziomieStacji,CisnienieNaPoziomieMorza,WielkoscTendencjiCisnienia,WysokoscOpadu,OkresPomiaruOpadu
0,DsTarnowek,2009-11-01 23:00:00,NaN,NaN,NaN,NaN,4.0000,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PmGdaPoWie01,2015-05-20 07:00:00,NaN,0.348464,54.1836,NaN,26.5142,NaN,4.08088,12146,2015-05-20,07:00:00,360.0,2.0,13.6,8.5,1011.4,1012.0,0.4,0.0,0.0
2,PmGdaWyzwo03,2015-05-20 07:00:00,NaN,0.299517,17.3023,NaN,10.0607,NaN,1.58940,12142,2015-05-20,07:00:00,0.0,1.0,12.3,8.3,1007.1,1012.5,0.8,0.0,0.0
3,PmGdaLeczk08,2015-05-20 07:00:00,NaN,0.394787,32.2047,46.242,13.2198,10.305,2.17913,12142,2015-05-20,07:00:00,0.0,1.0,12.3,8.3,1007.1,1012.5,0.8,0.0,0.0
4,PmGdaOstrz05,2015-05-20 07:00:00,NaN,0.264809,35.2662,57.725,14.5946,NaN,3.91802,12142,2015-05-20,07:00:00,0.0,1.0,12.3,8.3,1007.1,1012.5,0.8,0.0,0.0


In [46]:
statitics = ddf9.isnull().sum().reset_index().compute()
statitics.columns=['COLUMN NAME',"MISSING VALUES"]
statitics['TOTAL ROWS'] = ddf9.shape[0].compute()
statitics['% MISSING'] = round((statitics['MISSING VALUES']/statitics['TOTAL ROWS'])*100,2)
statitics

,COLUMN NAME,MISSING VALUES,TOTAL ROWS,% MISSING
0,Kod stacji_metadane,0,23889127,0.00
1,Czas pomiaru,0,23889127,0.00
2,C6H6,19937631,23889127,83.46
3,CO,13859866,23889127,58.02
4,NO2,5034175,23889127,21.07
5,O3,11905063,23889127,49.83
6,PM10,9396148,23889127,39.33
7,PM2.5,20321461,23889127,85.07
8,SO2,6444624,23889127,26.98
9,IDStacji_synop,0,23889127,0.00


In [47]:
ddf9.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,IDStacji_synop,Data,Godzina,KierunekWiatru,PredkoscWiatru,TemperaturaPowietrza,TemperaturaPunktuRosy,CisnienieNaPoziomieStacji,CisnienieNaPoziomieMorza,WielkoscTendencjiCisnienia,WysokoscOpadu,OkresPomiaruOpadu
0,DsTarnowek,2009-11-01 23:00:00,NaN,NaN,NaN,NaN,4.0000,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PmGdaPoWie01,2015-05-20 07:00:00,NaN,0.348464,54.1836,NaN,26.5142,NaN,4.08088,12146,2015-05-20,07:00:00,360.0,2.0,13.6,8.5,1011.4,1012.0,0.4,0.0,0.0
2,PmGdaWyzwo03,2015-05-20 07:00:00,NaN,0.299517,17.3023,NaN,10.0607,NaN,1.58940,12142,2015-05-20,07:00:00,0.0,1.0,12.3,8.3,1007.1,1012.5,0.8,0.0,0.0
3,PmGdaLeczk08,2015-05-20 07:00:00,NaN,0.394787,32.2047,46.242,13.2198,10.305,2.17913,12142,2015-05-20,07:00:00,0.0,1.0,12.3,8.3,1007.1,1012.5,0.8,0.0,0.0
4,PmGdaOstrz05,2015-05-20 07:00:00,NaN,0.264809,35.2662,57.725,14.5946,NaN,3.91802,12142,2015-05-20,07:00:00,0.0,1.0,12.3,8.3,1007.1,1012.5,0.8,0.0,0.0


In [48]:
ddf9=ddf9.replace({'IDStacji_synop': 0}, np.nan)
ddf9=client.persist(ddf9)

In [49]:
ddf9.dtypes

Kod stacji_metadane                   object
Czas pomiaru                  datetime64[ns]
C6H6                                 float64
CO                                   float64
NO2                                  float64
O3                                   float64
PM10                                 float64
PM2.5                                float64
SO2                                  float64
IDStacji_synop                         int64
Data                                  object
Godzina                               object
KierunekWiatru                        object
PredkoscWiatru                        object
TemperaturaPowietrza                  object
TemperaturaPunktuRosy                 object
CisnienieNaPoziomieStacji             object
CisnienieNaPoziomieMorza              object
WielkoscTendencjiCisnienia            object
WysokoscOpadu                         object
OkresPomiaruOpadu                     object
dtype: object

In [53]:
ddf9=ddf9.compute()